In [1]:
import os
import sys
sys.path.append("../../common/src")
sys.path.append("../src")
os.chdir("../src")

In [2]:
from services.agents.db_agent import run_db_agent, generate_sql_statement, execute_sql_statement, map_prompt_to_dataset

INFO: [config/config.py:55 - <module>()] Namespace File not found, setting job namespace as default
INFO: [config/config.py:105 - <module>()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:106 - <module>()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:107 - <module>()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:108 - <module>()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:109 - <module>()] ENABLE_TRUSS_LLAMA2 = True
INFO: [config/vector_store_config.py:40 - <module>()] Default vector store = [matching_engine]
INFO: [config/vector_store_config.py:49 - <module>()] PG_HOST = [127.0.0.1]
INFO: [config/vector_store_config.py:50 - <module>()] PG_DBNAME = [pgvector]
ERROR: [config/vector_store_config.py:77 - <module>()] Cannot connect to pgvector instance at 127.0.0.1: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: FATAL:  database "pgvector" does not exist

(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [3]:
prompt = "Which CDT codes are approved by medicaid?"
dataset, _ = map_prompt_to_dataset(prompt, None)
#prompt = "Can FQHCs bill CDT codes?"

In [4]:
dataset

'fqhc_medical_transactions'

In [6]:
statement, agent_logs = await generate_sql_statement(prompt, dataset)
statement

INFO: [agents/db_agent.py:122 - generate_sql_statement()] generating sql statement for dataset [fqhc_medical_transactions] prompt [Which CDT codes are approved by medicaid?] llm_type [None] db url [bigquery://gcp-mira-demo/fqhc_medical_transactions]
INFO: [agents/utils.py:44 - agent_executor_run_with_logs()] Agent process result: 

SELECT cdtCodeId, codeDescription FROM `CdtCode` WHERE medicaidApproved = TRUE ORDER BY cdtCodeId;
INFO: [agents/utils.py:45 - agent_executor_run_with_logs()] Agent process log: 



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: CdtCode, ClaimTransaction, FqhcProvider
Thought:The table that seems most relevant to the question about CDT codes approved by Medicaid is likely the CdtCode table. I should check the schema of this table to find out which columns might indicate whether a CDT code is approved by Medicaid.

Action: sql_db_schema
Action Input: CdtCode
Observation: 
CREATE TABLE `CdtCode` (
	`cdtCodeId` STRI

'SELECT cdtCodeId, codeDescription FROM `CdtCode` WHERE medicaidApproved = TRUE ORDER BY cdtCodeId;'

In [7]:
output = execute_sql_statement(statement, dataset)
output

INFO: [agents/db_agent.py:180 - execute_sql_statement()] running sql statement [SELECT cdtCodeId, codeDescription FROM `CdtCode` WHERE medicaidApproved = TRUE ORDER BY cdtCodeId;] for dataset [fqhc_medical_transactions] db url [bigquery://gcp-mira-demo/fqhc_medical_transactions]
INFO: [agents/db_agent.py:185 - execute_sql_statement()] got results [('D0120', 'periodic oral evaluation - established patient'), ('D0140', 'limited oral evaluation - problem focused'), ('D0150', 'comprehensive oral evaluation - new or established patient'), ('D0160', 'detailed and extensive oral evaluation - problem focused, by report'), ('D0170', 're-evaluation - limited, problem focused (established patient; not post-operative visit)'), ('D0180', 'comprehensive periodontal evaluation - new or established patient'), ('D0210', 'intraoral - complete series of radiographic images'), ('D0220', 'intraoral - periapical first radiographic image'), ('D0230', 'intraoral - periapical each additional radiographic image

Traceback (most recent call last):
  File "/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/urllib3/util/connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/socket.py", line 954, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/

Exception: HTTPConnectionPool(host='authentication', port=80): Max retries exceeded with url: /authentication/api/v1/sign-in/credentials (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x136f87ca0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
from services.agents.agent_tools import create_google_sheet
import datetime

In [ ]:
now = datetime.datetime.utcnow()
sheet_name = f"Dataset {dataset} Query {now}"

In [ ]:
sheet_name

In [ ]:
return_dict = output["data"]

In [ ]:
sheet_output = create_google_sheet(sheet_name,
                                   return_dict["columns"],
                                   return_dict["data"])